# Create Driver Dimension

In [0]:
import pyspark.sql.functions as F
from random import randint
from pyspark.sql.types import * 

In [0]:
df1=spark.read.format("delta").load("/mnt/Deltalake/Bronze/Trip_Transactions")

In [0]:
df1.createOrReplaceTempView("df1") 

In [0]:
df2=spark.sql('select distinct driver_id,driver_name from df1')

In [0]:
def driver_age(driver_id):
    return randint(18,65)
def driver_gender(driver_id):
    if randint(0,2)==0:
        return 'F'
    else:
        return "M"
    

In [0]:
spark.udf.register("driver_age", driver_age,IntegerType())
spark.udf.register("driver_gender", driver_gender,StringType())

Out[21]: <function __main__.driver_gender(driver_id)>

In [0]:
df2=df2.withColumn("driver_age",F.expr("driver_age(driver_id)"))
df2=df2.withColumn("driver_gender",F.expr("driver_gender(driver_id)"))



In [0]:
display(df2)

driver_id,driver_name,driver_age,driver_gender
A33,Umesh,24,F
A10,Amith,51,M
A23,Dhruv,65,M
A5,Sagar,30,F
A40,Ramesh,56,M
A1,Ram,56,M
A4,Aaditya,45,M
A28,Jack,24,F
A21,Arun,64,M
null,null,18,F


In [0]:
df2.repartition(1).write.save("/mnt/Deltalake/silver_Zone/driver_Dimension")